In [1]:
from functions_for_all_nutrients import *
from vitamin_a_and_zinc_functions import *

In [2]:
# Get absolute path of repo based on relative directory structure, so it should work for all users
import os.path
vivarium_research_lsff_path = os.path.abspath("..")
vivarium_research_lsff_path

'/ihme/homes/alibow/notebooks/vivarium_research_lsff'

In [3]:
username = !whoami
username

['alibow']

In [4]:
# Specify directory with saved GBD data so that it's not necessary to call get_draws on every run.

# Nathaniel has saved some .hdf files in this directory on /share
# You can specify a different directory if you have GBD data saved somewhere else.
# Or if you want to re-pull the data, you can change this to a nonexistent directory.
gbd_data_directory = '/share/scratch/users/ndbs/vivarium_lsff/gbd_data'
gbd_data_filepath = f'{gbd_data_directory}/multmodel_data.hdf'

In [5]:
index_cols=['location_id','sex_id','age_group_id']
age_group_ids = [2,3,4,5]
sex_ids = [1,2]
coverage_levels = [0.2,0.5,0.8,1]
years = [2021,2022,2023,2024,2025]

In [6]:
coverage_data_dir = vivarium_research_lsff_path + '/data_prep/outputs/waterfall_coverage_all_vehicles_oil_update_2021_04_16.csv'
# # Old version that may not work for all users:
# coverage_data_dir = f'/ihme/homes/{username[0]}/notebooks/' \
#     'vivarium_research_lsff/data_prep/outputs/lsff_input_coverage_data.csv'

In [7]:
location_ids = (list(
                pd.read_csv(coverage_data_dir)
                .location_id
                .unique()))
location_ids

[6,
 11,
 15,
 20,
 141,
 161,
 163,
 164,
 165,
 168,
 171,
 179,
 180,
 181,
 184,
 189,
 190,
 196,
 201,
 202,
 205,
 207,
 213,
 214,
 522]

# VITAMIN A

In [22]:
# vitamin A specific -- these should be replaced for other models
rei_id = 96
nutrient = 'vitamin a'
effective_fractions = [0, 0, (365 - (365 * 0.5)) / (365 - 28), 1]

In [45]:
vehicles = ['oil','zero oil',
           'industry oil','zero industry oil',
           'wheat flour', 'zero wheat flour',
           'industry wheat', 'zero industry wheat',
           'maize flour', 'zero maize flour']      

In [24]:
# define no fortification relative risk distribution
# vitamin a specific -- this should be replaced for other models

from numpy import log
from scipy.stats import norm, lognorm

# median and 0.975-quantile of lognormal distribution for RR
median = 2.22
q_975 = 5.26

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

mu = log(median) # mean of normal distribution for log(RR)
sigma = (log(q_975) - mu) / q_975_stdnorm # std dev of normal distribution for log(RR)

# Frozen lognormal distribution for RR, representing uncertainty in our effect size
# (s is the shape parameter)b
rr_distribution = lognorm(s=sigma, scale=median)

In [46]:
vehicles

['oil',
 'zero oil',
 'industry oil',
 'zero industry oil',
 'wheat flour',
 'zero wheat flour',
 'industry wheat',
 'zero industry wheat',
 'maize flour',
 'zero maize flour']

In [47]:
vitamin_a_baseline_coverage, vitamin_a_counterfactual_coverage = get_baseline_and_counterfactual_coverage(coverage_data_dir,
                                             location_ids,
                                             nutrient,
                                             vehicles,
                                             years,
                                             coverage_levels, 'U5')

Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [11] due to missing data
Excluded location IDs [11] due to missing data
Excluded location IDs [11] due to missing data
Excluded location IDs [11] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data


In [48]:
vitamin_a_baseline_effective_coverage = get_effective_vitamin_a_coverage(vitamin_a_baseline_coverage.set_index(['location_id','year','vehicle']), 
                                                               sex_ids,
                                                               age_group_ids,
                                                               effective_fractions,
                                                               years)
vitamin_a_counterfactual_effective_coverage = get_effective_vitamin_a_coverage(vitamin_a_counterfactual_coverage.set_index(['location_id','year','vehicle','coverage_level']), 
                                                               sex_ids,
                                                               age_group_ids,
                                                               effective_fractions,
                                                               years)

In [49]:
vitamin_a_rr_deficiency_nofort = generate_rr_deficiency_nofort_draws(mu, sigma, location_ids)
vitamin_a_rr_deficiency_nofort.mean(axis=1)

location_id
6      2.396239
11     2.396239
15     2.396239
20     2.396239
141    2.396239
161    2.396239
163    2.396239
164    2.396239
165    2.396239
168    2.396239
171    2.396239
179    2.396239
180    2.396239
181    2.396239
184    2.396239
189    2.396239
190    2.396239
196    2.396239
201    2.396239
202    2.396239
205    2.396239
207    2.396239
213    2.396239
214    2.396239
522    2.396239
dtype: float64

In [50]:
vitamin_a_dalys_hdf_key = '/vitamin_a_deficiency/dalys_attributable_bmgf_25_countries'
try:
    vitamin_a_dalys = pd.read_hdf(gbd_data_filepath, key=vitamin_a_dalys_hdf_key)
except (FileNotFoundError, KeyError) as e:
    print(e, "Calling get_draws...")
    vitamin_a_dalys = pull_deficiency_attributable_dalys(rei_id,
                                                  location_ids,
                                                  age_group_ids,
                                                  sex_ids,
                                                  index_cols)
else:
    # Stored data included all 25 locations, so filter to desired locations if try clause succeeded
    vitamin_a_dalys = vitamin_a_dalys.query("location_id in @location_ids")

vitamin_a_dalys.head()

draw_0       draw_1      draw_10  \
location_id sex_id age_group_id                                          
6           1      2                6.518769    13.105468     3.094651   
                   3              137.319706   160.767128    94.890071   
                   4             2655.803809  1851.342551  1589.046172   
                   5             6014.654573  5196.950929  5416.861679   
            2      2                1.257434     0.399007     0.553913   

                                    draw_100     draw_101     draw_102  \
location_id sex_id age_group_id                                          
6           1      2               11.038021     4.401997    11.975454   
                   3               90.369198    60.432847    55.529898   
                   4             1599.004574  1346.298841  1256.386415   
                   5             5331.341355  3854.742854  3731.005037   
            2      2                1.023849     0.758328     2.072954   

                                    draw_103     draw_104     draw_105  \
location_id sex_id age_group_id                                          
6           1      2                7.494744     8.345612     6.911725   
                   3               90.291739   271.638485    98.426889   
                   4             2277.158107  1945.723118  1648.834016   
                   5             5998.981182  5148.786063  6713.467238   
            2      2                1.215976     0.498824     1.484751   

                                    draw_106  ...     draw_990     draw_991  \
location_id sex_id age_group_id               ...                             
6           1      2                3.246637  ...     7.744664     6.914776   
                   3              103.576291  ...    46.290755   114.677909   
                   4             1727.435654  ...  1401.487949  2418.174399   
                   5             4967.112706  ...  4836.679039  7762.494425   
            2      2                0.292424  ...     0.617292     2.122949   

                                    draw_992     draw_993     draw_994  \
location_id sex_id age_group_id                                          
6           1      2               16.271661     9.998869     8.958759   
                   3              162.199784    44.200716   132.494285   
                   4             1694.596125  1575.049825  1867.622230   
                   5             9917.792849  5999.923366  5959.441779   
            2      2                0.496779     0.652097     1.054172   

                                    draw_995     draw_996     draw_997  \
location_id sex_id age_group_id                                          
6           1      2               10.438845     4.813849     2.642819   
                   3               77.170521   111.370798    50.678223   
                   4             2990.603818   965.710245  1477.468702   
                   5             6283.476586  2741.025158  4291.649161   
            2      2                0.565635     0.495569     0.357408   

                                    draw_998     draw_999  
location_id sex_id age_group_id                            
6           1      2                6.619699     4.775778  
                   3              179.346982   140.162456  
                   4             1660.723099  1682.522484  
                   5             4393.900720  6320.589774  
            2      2                0.691123     0.653190  

[5 rows x 1000 columns]

In [51]:
vitamin_a_paf_deficiency_nofort = calculate_paf_deficiency_nofort(
                                    vitamin_a_rr_deficiency_nofort, 
                                    vitamin_a_baseline_effective_coverage)
vitamin_a_paf_deficiency_nofort.head()

draws                                                    draw_0    draw_1  \
location_id sex_id age_group_id year vehicle                                
6           1      2            2021 industry oil       0.78595  0.447022   
                                     industry wheat     0.78595  0.447022   
                                     oil                0.78595  0.447022   
                                     wheat flour        0.78595  0.447022   
                                     zero industry oil  0.78595  0.447022   

draws                                                    draw_10  draw_100  \
location_id sex_id age_group_id year vehicle                                 
6           1      2            2021 industry oil       0.406811  0.340996   
                                     industry wheat     0.406811  0.340996   
                                     oil                0.406811  0.340996   
                                     wheat flour        0.406811  0.340996   
                                     zero industry oil  0.406811  0.340996   

draws                                                   draw_101  draw_102  \
location_id sex_id age_group_id year vehicle                                 
6           1      2            2021 industry oil       0.617978  0.620854   
                                     industry wheat     0.617978  0.620854   
                                     oil                0.617978  0.620854   
                                     wheat flour        0.617978  0.620854   
                                     zero industry oil  0.617978  0.620854   

draws                                                   draw_103  draw_104  \
location_id sex_id age_group_id year vehicle                                 
6           1      2            2021 industry oil       0.149863  0.636347   
                                     industry wheat     0.149863  0.636347   
                                     oil                0.149863  0.636347   
                                     wheat flour        0.149863  0.636347   
                                     zero industry oil  0.149863  0.636347   

draws                                                   draw_105  draw_106  \
location_id sex_id age_group_id year vehicle                                 
6           1      2            2021 industry oil       0.421241  0.759597   
                                     industry wheat     0.421241  0.759597   
                                     oil                0.421241  0.759597   
                                     wheat flour        0.421241  0.759597   
                                     zero industry oil  0.421241  0.759597   

draws                                                   ...  draw_990  \
location_id sex_id age_group_id year vehicle            ...             
6           1      2            2021 industry oil       ...  0.507946   
                                     industry wheat     ...  0.507946   
                                     oil                ...  0.507946   
                                     wheat flour        ...  0.507946   
                                     zero industry oil  ...  0.507946   

draws                                                   draw_991  draw_992  \
location_id sex_id age_group_id year vehicle                                 
6           1      2            2021 industry oil       0.455066  0.740104   
                                     industry wheat     0.455066  0.740104   
                                     oil                0.455066  0.740104   
                                     wheat flour        0.455066  0.740104   
                                     zero industry oil  0.455066  0.740104   

draws                                                   draw_993  draw_994  \
location_id sex_id age_group_id year vehicle                                 
6           1      2            2021 industry oil       0

In [52]:
vitamin_a_pif_deficiency_nofort = calculate_pif_deficiency_nofort(
                                    vitamin_a_paf_deficiency_nofort,
                                    vitamin_a_baseline_effective_coverage,
                                    vitamin_a_counterfactual_effective_coverage)
vitamin_a_pif_deficiency_nofort.head()

draws                                                               draw_0  \
location_id sex_id age_group_id year vehicle        coverage_level           
6           1      2            2021 industry oil   0.2                0.0   
                                                    0.5                0.0   
                                                    0.8                0.0   
                                                    1.0                0.0   
                                     industry wheat 0.2                0.0   

draws                                                               draw_1  \
location_id sex_id age_group_id year vehicle        coverage_level           
6           1      2            2021 industry oil   0.2                0.0   
                                                    0.5                0.0   
                                                    0.8                0.0   
                                                    1.0                0.0   
                                     industry wheat 0.2                0.0   

draws                                                               draw_10  \
location_id sex_id age_group_id year vehicle        coverage_level            
6           1      2            2021 industry oil   0.2                 0.0   
                                                    0.5                 0.0   
                                                    0.8                 0.0   
                                                    1.0                 0.0   
                                     industry wheat 0.2                 0.0   

draws                                                               draw_100  \
location_id sex_id age_group_id year vehicle        coverage_level             
6           1      2            2021 industry oil   0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     industry wheat 0.2                  0.0   

draws                                                               draw_101  \
location_id sex_id age_group_id year vehicle        coverage_level             
6           1      2            2021 industry oil   0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     industry wheat 0.2                  0.0   

draws                                                               draw_102  \
location_id sex_id age_group_id year vehicle        coverage_level             
6           1      2            2021 industry oil   0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     industry wheat 0.2                  0.0   

draws                                                               draw_103  \
location_id sex_id age_group_id year vehicle        coverage_level             
6           1      2            2021 industry oil   0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     industry wheat 0.2                  0.0   

draws                                                               draw_104  \
location_id sex_id age_group

In [53]:
vitamin_a_exposure = get_draws(gbd_round_id=6,
                              year_id=2019,
                              location_id=location_ids,
                              sex_id=sex_ids,
                              age_group_id=age_group_ids,
                              gbd_id_type='rei_id',
                              gbd_id=96,
                              source='exposure',
                              decomp_step='step4')

vitamin_a_exp = (vitamin_a_exposure.loc[vitamin_a_exposure.parameter=='cat1']
                 .set_index(['location_id','sex_id','age_group_id']))
vitamin_a_exp = vitamin_a_exp.drop(columns=[c for c in vitamin_a_exp.columns if 'draw' not in c])

vitamin_a_exp = vitamin_a_exp - (vitamin_a_exp * vitamin_a_pif_deficiency_nofort)

vitamin_a_exp = vitamin_a_exp.reset_index().merge(get_population(gbd_round_id=6,
                    location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    year_id=2019,
                    decomp_step='step4'), on=['location_id','sex_id','age_group_id'])

for i in list(range(0,1000)):
    vitamin_a_exp[f'draw_{i}'] = vitamin_a_exp[f'draw_{i}'] * vitamin_a_exp.population
    
vitamin_a_exp = vitamin_a_exp.groupby(['location_id','vehicle','year','coverage_level']).sum()

for i in list(range(0,1000)):
    vitamin_a_exp[f'draw_{i}'] = vitamin_a_exp[f'draw_{i}'] / vitamin_a_exp.population * 100
    
vitamin_a_exp = (vitamin_a_exp.drop(columns=[c for c in vitamin_a_exp.columns if 'draw' not in c])
                 .reset_index())
vitamin_a_exp = vitamin_a_exp.loc[vitamin_a_exp.year==2025]#.drop(columns='year')
vitamin_a_exp['nutrient'] = 'vitamin A'

vitamin_a_exp

,location_id,vehicle,year,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,nutrient
16,6,industry oil,2025,0.2,2.365160,3.592416,3.466884,5.368348,3.093504,4.197759,...,4.060686,5.111956,3.433926,4.830706,4.672589,3.219522,1.929398,2.368575,5.454848,vitamin A
17,6,industry oil,2025,0.5,2.137193,3.444156,3.355659,5.222810,2.915570,3.923957,...,3.916059,4.626128,3.071291,4.660976,4.447962,3.036962,1.860803,2.305625,5.281915,vitamin A
18,6,industry oil,2025,0.8,1.909226,3.295895,3.244434,5.077272,2.737636,3.650154,...,3.771432,4.140299,2.708657,4.491246,4.223335,2.854402,1.792207,2.242675,5.108983,vitamin A
19,6,industry oil,2025,1.0,1.757248,3.197055,3.170284,4.980247,2.619013,3.467619,...,3.675014,3.816413,2.466901,4.378093,4.073583,2.732696,1.746477,2.200709,4.993695,vitamin A
36,6,industry wheat,2025,0.2,2.390267,3.584626,3.399520,5.320748,3.036323,4.147948,...,3.977227,5.111636,3.439996,4.793490,4.633254,3.144121,1.891141,2.349321,5.367464,vitamin A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4579,522,zero oil,2025,1.0,35.189250,23.670043,23.709344,22.833109,47.811956,45.846108,...,24.570509,47.748504,50.844488,37.374341,35.202605,46.416811,28.643643,24.488204,28.715709,vitamin A
4596,522,zero wheat flour,2025,0.2,15.358419,15.906656,16.890842,17.162246,24.407260,23.678135,...,16.706674,21.983471,22.368464,25.473018,20.259974,23.617950,18.729026,17.297064,20.929467,vitamin A
4597,522,zero wheat flour,2025,0.5,15.553499,16.081441,17.012209,17.291700,24.725981,23.937912,...,16.853555,22.482955,22.766586,25.756417,20.473307,23.962418,18.921956,17.409529,21.120883,vitamin A
4598,522,zero wheat flour,2025,0.8,15.748579,16.256226,17.133576,17.421154,25.044702,24.197688,...,17.000436,22.982438,23.164707,26.039817,20.686640,24.306885,19.114886,17.521993,21.312298,vitamin A


In [54]:
vitamin_a_exp.to_pickle('results_raw/vitamin_a_deficiency_exposure.pkl')

In [55]:
vitamin_a_overall_pifs, vitamin_a_daly_reduction_counts = calculate_final_pifs_and_daly_reductions(
                                                    vitamin_a_pif_deficiency_nofort,
                                                    vitamin_a_dalys,
                                                    coverage_levels, years)

In [56]:
vitamin_a_daly_reduction_rates = calculate_rates(vitamin_a_daly_reduction_counts, 
                                            location_ids, 
                                            age_group_ids, 
                                            sex_ids)

In [57]:
vitamin_a_overall_pifs['measure'] = 'pif'
vitamin_a_daly_reduction_counts['measure'] = 'counts_averted'
vitamin_a_daly_reduction_rates['measure'] = 'rates_averted'
vitamin_a_results = pd.concat([vitamin_a_overall_pifs.reset_index(),
                         vitamin_a_daly_reduction_counts.reset_index(),
                         vitamin_a_daly_reduction_rates.reset_index()],
                        ignore_index=True)
vitamin_a_results.to_pickle('results_raw/vitamin_a.pkl')
vitamin_a_results

draws,location_id,year,vehicle,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,measure
0,6,2021,industry oil,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
1,6,2021,industry oil,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
2,6,2021,industry oil,0.8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
3,6,2021,industry oil,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
4,6,2021,industry wheat,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,pif
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13795,522,2025,zero oil,1.0,-337.215679,-168.189785,-70.773886,-136.027080,-93.911314,-120.817961,...,-130.236523,-513.497547,-521.609837,-174.791990,-159.084363,-141.916252,-118.769152,-119.620333,-66.569674,rates_averted
13796,522,2025,zero wheat flour,0.2,-2.275681,-2.536817,-0.835231,-2.105451,-0.881401,-0.966978,...,-1.624475,-6.674980,-4.975057,-2.802472,-1.543757,-1.506933,-1.553347,-1.289021,-1.091587,rates_averted
13797,522,2025,zero wheat flour,0.5,-5.689202,-6.342042,-2.088077,-5.263629,-2.203502,-2.417445,...,-4.061189,-16.687451,-12.437643,-7.006180,-3.859393,-3.767331,-3.883368,-3.222551,-2.728968,rates_averted
13798,522,2025,zero wheat flour,0.8,-9.102722,-10.147268,-3.340923,-8.421806,-3.525604,-3.867912,...,-6.497902,-26.699922,-19.900230,-11.209888,-6.175028,-6.027730,-6.213389,-5.156082,-4.366349,rates_averted


# ZINC

In [23]:
# ZINC
rei_id = 97
nutrient = 'zinc'
#vehicles = ['wheat flour','maize flour','zero wheat flour','zero maize flour']
effective_fractions = [0,0,0,1]

In [24]:
vehicles = ['wheat flour', 'zero wheat flour',
           'industry wheat', 'zero industry wheat',
           'maize flour', 'zero maize flour']

In [25]:
# define no fortification relative risk distribution
# vitamin a specific -- this should be replaced for other models

from numpy import log
from scipy.stats import norm, lognorm

# median and 0.975-quantile of lognormal distribution for RR
median = 0.47
q_975 = 0.69

# 0.975-quantile of standard normal distribution (=1.96, approximately)
q_975_stdnorm = norm().ppf(0.975)

mu = log(median) # mean of normal distribution for log(RR)
sigma = (log(q_975) - mu) / q_975_stdnorm # std dev of normal distribution for log(RR)

# Frozen lognormal distribution for RR, representing uncertainty in our effect size
# (s is the shape parameter)
rr_distribution = lognorm(s=sigma, scale=median)

In [26]:
zinc_baseline_coverage = pd.read_pickle('results_raw/baseline_coverage_data_u5.pkl')
zinc_baseline_coverage = (zinc_baseline_coverage.loc[zinc_baseline_coverage.nutrient==nutrient]
                              .loc[zinc_baseline_coverage.vehicle.isin(vehicles)]
                              .drop(columns='nutrient'))

zinc_counterfactual_coverage = pd.read_pickle('results_raw/counterfactual_coverage_data_u5.pkl')
zinc_counterfactual_coverage = (zinc_counterfactual_coverage.loc[zinc_counterfactual_coverage.nutrient==nutrient]
                              .loc[zinc_counterfactual_coverage.vehicle.isin(vehicles)]
                                    .drop(columns='nutrient'))

zinc_baseline_coverage.head()

,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,draw_103,draw_104,draw_105,draw_106,...,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,location_id,year,vehicle
1869,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2021.0,wheat flour
1870,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2022.0,wheat flour
1871,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2023.0,wheat flour
1872,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2024.0,wheat flour
1873,0.048558,0.040897,0.057464,0.055794,0.078894,0.069322,0.066442,0.074779,0.040128,0.051428,...,0.060894,0.038659,0.063932,0.084612,0.040307,0.07357,0.062551,6,2025.0,wheat flour


In [27]:
zinc_baseline_coverage, zinc_counterfactual_coverage = get_baseline_and_counterfactual_coverage(coverage_data_dir,
                                             location_ids,
                                             nutrient,
                                             vehicles,
                                             years,
                                             coverage_levels, 'U5')

Excluded location IDs [] due to missing data
Excluded [11]/zinc/wheat flour due to impossible logical values
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data
Excluded location IDs [] due to missing data


In [28]:
zinc_rr_deficiency_nofort = (1 / generate_rr_deficiency_nofort_draws(mu, sigma, location_ids))
zinc_rr_deficiency_nofort.mean(axis=1)

location_id
168    2.178029
161    2.178029
201    2.178029
205    2.178029
202    2.178029
6      2.178029
171    2.178029
141    2.178029
179    2.178029
207    2.178029
163    2.178029
11     2.178029
180    2.178029
181    2.178029
184    2.178029
15     2.178029
164    2.178029
213    2.178029
214    2.178029
165    2.178029
196    2.178029
522    2.178029
190    2.178029
189    2.178029
20     2.178029
dtype: float64

In [29]:
zinc_dalys_hdf_key = '/zinc_deficiency/dalys_attributable_bmgf_25_countries'
try:
    zinc_dalys = pd.read_hdf(gbd_data_filepath, key=zinc_dalys_hdf_key)
except (FileNotFoundError, KeyError) as e:
    print(e, "Calling get_draws...")
    zinc_dalys = pull_deficiency_attributable_dalys(rei_id,
                                              location_ids,
                                              age_group_ids,
                                              sex_ids,
                                              index_cols)
else:
    # Stored data included all 25 locations, so filter to desired locations if try clause succeeded
    zinc_dalys = zinc_dalys.query("location_id in @location_ids")

zinc_dalys.head()

draw_0      draw_1     draw_10  \
location_id sex_id age_group_id                                        
6           1      5               55.806889    0.000000   29.211177   
            2      5               37.542436    0.000000   21.781172   
11          1      5             1057.924493  369.707946  517.346114   
            2      5              954.532812  197.702087  436.339271   
15          1      5              313.163965   18.266017   79.234558   

                                    draw_100    draw_101    draw_102  \
location_id sex_id age_group_id                                        
6           1      5               91.031178   45.761789   29.550910   
            2      5               86.086341   38.032330   24.744728   
11          1      5             1121.865646  383.960301  326.811523   
            2      5             1209.191167  371.705455  208.350528   
15          1      5              128.724069   27.397756   42.252433   

                                   draw_103     draw_104     draw_105  \
location_id sex_id age_group_id                                         
6           1      5              11.862744    39.110218   142.270449   
            2      5               8.679425    33.560425   124.455756   
11          1      5             290.876450  1234.791858  1980.744828   
            2      5             464.778852  1270.639991  1371.653872   
15          1      5              45.034578   168.170188   167.670791   

                                   draw_106  ...     draw_990     draw_991  \
location_id sex_id age_group_id              ...                             
6           1      5              27.921471  ...    70.827393   154.927527   
            2      5              18.714235  ...    63.459794   123.302919   
11          1      5             928.505912  ...  1295.128773  2676.259203   
            2      5             665.427560  ...  1074.683581  2096.088331   
15          1      5             189.359412  ...   132.672371   285.220362   

                                    draw_992     draw_993     draw_994  \
location_id sex_id age_group_id                                          
6           1      5               29.528407   146.462447   219.592170   
            2      5               20.750247   115.160657   196.325055   
11          1      5             1164.434543  1504.802755  3333.240053   
            2      5              897.944516   993.337161  2816.770108   
15          1      5              102.390704   220.484294   547.323432   

                                    draw_995     draw_996     draw_997  \
location_id sex_id age_group_id                                          
6           1      5              165.562467    62.928907    32.051667   
            2      5              154.511623    51.370814    35.211259   
11          1      5             2298.076536  1813.060392  1378.863465   
            2      5             2380.433080  1282.128241   957.580283   
15          1      5              387.181049   198.262244    86.842222   

                                    draw_998     draw_999  
location_id sex_id age_group_id                            
6           1      5               77.849291   149.179710  
            2      5               63.368359   129.279112  
11          1      5             1424.566740  1897.192422  
            2      5             1063.576027  1460.532078  
15          1      5              112.237688   146.724520  

[5 rows x 1000 columns]

In [30]:
zinc_baseline_effective_coverage = apply_age_related_effective_coverage_restrictions(
                                                                zinc_baseline_coverage.set_index([c for c in zinc_baseline_coverage.columns if 'draw' not in c]),
                                                                sex_ids,
                                                                age_group_ids,
                                                                effective_fractions)
zinc_counterfactual_effective_coverage = apply_age_related_effective_coverage_restrictions(
                                                                zinc_counterfactual_coverage.set_index([c for c in zinc_counterfactual_coverage.columns if 'draw' not in c]),
                                                                sex_ids,
                                                                age_group_ids,
                                                                effective_fractions)

In [31]:
zinc_paf_deficiency_nofort = calculate_paf_deficiency_nofort(
                                    zinc_rr_deficiency_nofort, 
                                    zinc_baseline_effective_coverage)
zinc_paf_deficiency_nofort.head()

draws                                                       draw_0    draw_1  \
location_id age_group_id sex_id year vehicle                                   
6           2            1      2021 industry wheat       0.345476  0.571001   
                                     wheat flour          0.345476  0.571001   
                                     zero industry wheat  0.345476  0.571001   
                                     zero wheat flour     0.345476  0.571001   
                                2022 industry wheat       0.345476  0.571001   

draws                                                      draw_10  draw_100  \
location_id age_group_id sex_id year vehicle                                   
6           2            1      2021 industry wheat       0.584197  0.603221   
                                     wheat flour          0.584197  0.603221   
                                     zero industry wheat  0.584197  0.603221   
                                     zero wheat flour     0.584197  0.603221   
                                2022 industry wheat       0.584197  0.603221   

draws                                                     draw_101  draw_102  \
location_id age_group_id sex_id year vehicle                                   
6           2            1      2021 industry wheat       0.494235   0.49253   
                                     wheat flour          0.494235   0.49253   
                                     zero industry wheat  0.494235   0.49253   
                                     zero wheat flour     0.494235   0.49253   
                                2022 industry wheat       0.494235   0.49253   

draws                                                     draw_103  draw_104  \
location_id age_group_id sex_id year vehicle                                   
6           2            1      2021 industry wheat       0.645743  0.483019   
                                     wheat flour          0.645743  0.483019   
                                     zero industry wheat  0.645743  0.483019   
                                     zero wheat flour     0.645743  0.483019   
                                2022 industry wheat       0.645743  0.483019   

draws                                                     draw_105  draw_106  \
location_id age_group_id sex_id year vehicle                                   
6           2            1      2021 industry wheat       0.579614  0.378442   
                                     wheat flour          0.579614  0.378442   
                                     zero industry wheat  0.579614  0.378442   
                                     zero wheat flour     0.579614  0.378442   
                                2022 industry wheat       0.579614  0.378442   

draws                                                     ...  draw_990  \
location_id age_group_id sex_id year vehicle              ...             
6           2            1      2021 industry wheat       ...  0.548122   
                                     wheat flour          ...  0.548122   
                                     zero industry wheat  ...  0.548122   
                                     zero wheat flour     ...  0.548122   
                                2022 industry wheat       ...  0.548122   

draws                                                     draw_991  draw_992  \
location_id age_group_id sex_id year vehicle                                   
6           2            1      2021 industry wheat       0.568194  0.399642   
                                     wheat flour          0.568194  0.399642   
                                     zero industry wheat  0.568194  0.399642   
                                     zero wheat flour     0.568194  0.399642   
                                2022 industry wheat       0.568194  0.399642   

draws                                                     draw_993  draw_994  \
location_id age_group_id sex

In [32]:
zinc_pif_deficiency_nofort = calculate_pif_deficiency_nofort(
                                    zinc_paf_deficiency_nofort,
                                    zinc_baseline_effective_coverage,
                                    zinc_counterfactual_effective_coverage)
zinc_pif_deficiency_nofort.head()

draws                                                               draw_0  \
location_id age_group_id sex_id year vehicle        coverage_level           
6           2            1      2021 industry wheat 0.2                0.0   
                                                    0.5                0.0   
                                                    0.8                0.0   
                                                    1.0                0.0   
                                     wheat flour    0.2                0.0   

draws                                                               draw_1  \
location_id age_group_id sex_id year vehicle        coverage_level           
6           2            1      2021 industry wheat 0.2                0.0   
                                                    0.5                0.0   
                                                    0.8                0.0   
                                                    1.0                0.0   
                                     wheat flour    0.2                0.0   

draws                                                               draw_10  \
location_id age_group_id sex_id year vehicle        coverage_level            
6           2            1      2021 industry wheat 0.2                 0.0   
                                                    0.5                 0.0   
                                                    0.8                 0.0   
                                                    1.0                 0.0   
                                     wheat flour    0.2                 0.0   

draws                                                               draw_100  \
location_id age_group_id sex_id year vehicle        coverage_level             
6           2            1      2021 industry wheat 0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     wheat flour    0.2                  0.0   

draws                                                               draw_101  \
location_id age_group_id sex_id year vehicle        coverage_level             
6           2            1      2021 industry wheat 0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     wheat flour    0.2                  0.0   

draws                                                               draw_102  \
location_id age_group_id sex_id year vehicle        coverage_level             
6           2            1      2021 industry wheat 0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     wheat flour    0.2                  0.0   

draws                                                               draw_103  \
location_id age_group_id sex_id year vehicle        coverage_level             
6           2            1      2021 industry wheat 0.2                  0.0   
                                                    0.5                  0.0   
                                                    0.8                  0.0   
                                                    1.0                  0.0   
                                     wheat flour    0.2                  0.0   

draws                                                               draw_104  \
location_id age_group_id sex

In [33]:
zinc_overall_pifs, zinc_daly_reduction_counts = calculate_final_pifs_and_daly_reductions(
                                                    zinc_pif_deficiency_nofort,
                                                    zinc_dalys,
                                                    coverage_levels, years)

In [34]:
zinc_daly_reduction_rates = calculate_rates(zinc_daly_reduction_counts, 
                                            location_ids, 
                                            age_group_ids, 
                                            sex_ids)

In [35]:
zinc_overall_pifs['measure'] = 'pif'
zinc_daly_reduction_counts['measure'] = 'counts_averted'
zinc_daly_reduction_rates['measure'] = 'rates_averted'
zinc_results = pd.concat([zinc_overall_pifs.reset_index(),
                         zinc_daly_reduction_counts.reset_index(),
                         zinc_daly_reduction_rates.reset_index()],
                        ignore_index=True)
zinc_results.to_pickle('results_raw/zinc.pkl')

In [36]:
zinc_exposure = get_draws(gbd_round_id=6,
                              year_id=2019,
                              location_id=location_ids,
                              sex_id=sex_ids,
                              age_group_id=age_group_ids,
                              gbd_id_type='rei_id',
                              gbd_id=97,
                              source='exposure',
                              decomp_step='step4')

zinc_exp = (zinc_exposure.loc[zinc_exposure.parameter=='cat1']
                 .set_index(['location_id','age_group_id','sex_id']))
zinc_exp = zinc_exp.drop(columns=[c for c in zinc_exp.columns if 'draw' not in c])

zinc_exp = zinc_exp - (zinc_exp * zinc_pif_deficiency_nofort)

zinc_exp = zinc_exp.reset_index().merge(get_population(gbd_round_id=6,
                    location_id=location_ids,
                    sex_id=sex_ids,
                    age_group_id=age_group_ids,
                    year_id=2019,
                    decomp_step='step4'), on=['location_id','sex_id','age_group_id'])

for i in list(range(0,1000)):
    zinc_exp[f'draw_{i}'] = zinc_exp[f'draw_{i}'] * zinc_exp.population
    
zinc_exp = zinc_exp.groupby(['location_id','vehicle','year','coverage_level']).sum()

for i in list(range(0,1000)):
    zinc_exp[f'draw_{i}'] = zinc_exp[f'draw_{i}'] / zinc_exp.population * 100
    
zinc_exp = (zinc_exp.drop(columns=[c for c in zinc_exp.columns if 'draw' not in c])
                 .reset_index())
zinc_exp = zinc_exp.loc[zinc_exp.year==2025]
zinc_exp['nutrient'] = 'zinc'

zinc_exp

,location_id,vehicle,year,coverage_level,draw_0,draw_1,draw_10,draw_100,draw_101,draw_102,...,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999,nutrient
16,6,industry wheat,2025,0.2,0.362620,0.000000,0.228163,0.568770,0.557028,0.445450,...,1.480318,0.110511,0.735176,1.208414,1.572021,0.537372,0.261757,0.663658,1.747535,zinc
17,6,industry wheat,2025,0.5,0.348499,0.000000,0.210892,0.525060,0.512962,0.413534,...,1.355340,0.103913,0.702144,1.136232,1.457692,0.494354,0.246171,0.603744,1.606226,zinc
18,6,industry wheat,2025,0.8,0.334378,0.000000,0.193621,0.481350,0.468895,0.381618,...,1.230362,0.097316,0.669112,1.064051,1.343363,0.451335,0.230585,0.543831,1.464917,zinc
19,6,industry wheat,2025,1.0,0.324964,0.000000,0.182107,0.452210,0.439517,0.360341,...,1.147043,0.092918,0.647091,1.015930,1.267143,0.422656,0.220195,0.503889,1.370710,zinc
36,6,wheat flour,2025,0.2,0.362598,0.000000,0.227301,0.566955,0.551045,0.442204,...,1.467234,0.109735,0.732990,1.213139,1.562912,0.530769,0.262585,0.656613,1.737130,zinc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2639,522,zero maize flour,2025,1.0,26.849188,20.926396,25.044909,26.460801,23.759140,21.096823,...,33.956273,25.187962,25.873864,32.577044,35.744054,31.812508,28.074425,28.174493,32.640367,zinc
2656,522,zero wheat flour,2025,0.2,26.720435,20.350234,24.603903,25.754839,23.417735,20.781646,...,32.937597,24.928249,25.348052,31.872200,35.146989,30.775833,26.957238,27.570728,31.958541,zinc
2657,522,zero wheat flour,2025,0.5,26.845664,20.594447,24.856823,26.115388,23.738205,21.013423,...,33.231572,25.064637,25.475351,32.418586,35.502547,31.134671,27.372620,27.967981,32.182080,zinc
2658,522,zero wheat flour,2025,0.8,26.970894,20.838661,25.109743,26.475936,24.058676,21.245201,...,33.525547,25.201025,25.602649,32.964973,35.858105,31.493509,27.788003,28.365233,32.405619,zinc


In [37]:
zinc_exp.to_pickle('results_raw/zinc_deficiency_exposure.pkl')